In [4]:
# 추천 시스템
# 영화

import pandas as pd
df = pd.read_csv('./ml-100k/u.data',sep='\t',header=None)
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']
df

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [8]:
# user id, item id, rating을 이용하여 예측

df.groupby(['rating'])[['user_id']].count()
# rating : 1 -> 6110 명 평가

,user_id
rating,
1,6110
2,11370
3,27145
4,34174
5,21201


In [10]:
df.groupby(['item_id'])[['user_id']].count()
# item_id 1번에 대해 452명이 평가

,user_id
item_id,
1,452
2,131
3,90
4,209
5,86
...,...
1678,1
1679,1
1680,1


In [15]:
n_users = df.user_id.unique().shape[0]
n_users

943

In [17]:
n_items = df.item_id.unique().shape[0]
n_items

1682

In [19]:
import numpy as np
ratings = np.zeros((n_users, n_items))
ratings.shape
# users x items
# rating[0][13] => 첫번째 사용자의 14번째 영화에 대한 평가

(943, 1682)

In [23]:
list(df.itertuples())

[Pandas(Index=0, user_id=196, item_id=242, rating=3, timestamp=881250949),
 Pandas(Index=1, user_id=186, item_id=302, rating=3, timestamp=891717742),
 Pandas(Index=2, user_id=22, item_id=377, rating=1, timestamp=878887116),
 Pandas(Index=3, user_id=244, item_id=51, rating=2, timestamp=880606923),
 Pandas(Index=4, user_id=166, item_id=346, rating=1, timestamp=886397596),
 Pandas(Index=5, user_id=298, item_id=474, rating=4, timestamp=884182806),
 Pandas(Index=6, user_id=115, item_id=265, rating=2, timestamp=881171488),
 Pandas(Index=7, user_id=253, item_id=465, rating=5, timestamp=891628467),
 Pandas(Index=8, user_id=305, item_id=451, rating=3, timestamp=886324817),
 Pandas(Index=9, user_id=6, item_id=86, rating=3, timestamp=883603013),
 Pandas(Index=10, user_id=62, item_id=257, rating=2, timestamp=879372434),
 Pandas(Index=11, user_id=286, item_id=1014, rating=5, timestamp=879781125),
 Pandas(Index=12, user_id=200, item_id=222, rating=5, timestamp=876042340),
 Pandas(Index=13, user_id=2

In [24]:
for row in df.itertuples():
    # [user_id,item_id] = rating
    ratings[row[1]-1, row[2]-1] = row[3]

In [25]:
ratings.shape

(943, 1682)

In [26]:
ratings
# user_id : 1, item_id : 1 의 평가는 5임을 알 수 있음

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [27]:
# rating을 train, test로 분리
from sklearn.model_selection import train_test_split

train_rating, test_rating = train_test_split(ratings, test_size=0.3, random_state=42)
train_rating.shape, test_rating.shape
# 행 : user_id, 열 : item_id

((660, 1682), (283, 1682))

In [35]:
from sklearn.metrics.pairwise import cosine_distances

# 코사인 유사도로 사용자간의 유사도를 구한다.
# cosine_distance : 행x행 사이즈로 변한다.
train_cosine = cosine_distances(train_rating)
train_cosine, train_cosine.shape

(array([[0.        , 0.82551651, 0.81990246, ..., 0.86668541, 0.82304407,
         0.78117519],
        [0.82551651, 0.        , 0.92006903, ..., 0.92824192, 0.90447378,
         0.94487345],
        [0.81990246, 0.92006903, 0.        , ..., 0.9808264 , 0.97766615,
         0.89689215],
        ...,
        [0.86668541, 0.92824192, 0.9808264 , ..., 0.        , 0.95146572,
         0.94857492],
        [0.82304407, 0.90447378, 0.97766615, ..., 0.95146572, 0.        ,
         0.8801978 ],
        [0.78117519, 0.94487345, 0.89689215, ..., 0.94857492, 0.8801978 ,
         0.        ]]),
 (660, 660))

In [31]:
# 거리 = 1- 유사도
# 유사도와 거리는 반비례, 유사도가 높을수록 거리는 멈

traing_distance = 1 - cosine_distances(train_rating)
traing_distance


array([[1.        , 0.17448349, 0.18009754, ..., 0.13331459, 0.17695593,
        0.21882481],
       [0.17448349, 1.        , 0.07993097, ..., 0.07175808, 0.09552622,
        0.05512655],
       [0.18009754, 0.07993097, 1.        , ..., 0.0191736 , 0.02233385,
        0.10310785],
       ...,
       [0.13331459, 0.07175808, 0.0191736 , ..., 1.        , 0.04853428,
        0.05142508],
       [0.17695593, 0.09552622, 0.02233385, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.21882481, 0.05512655, 0.10310785, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [33]:
traing_distance.shape, train_rating.shape

((660, 660), (660, 1682))

In [38]:
np.array([np.abs(traing_distance).sum(axis=1)]).T

array([[152.96763289],
       [ 89.87733739],
       [ 55.09121397],
       [103.16539004],
       [130.78394185],
       [101.87022277],
       [132.15773446],
       [ 79.71256965],
       [ 84.07171086],
       [162.34663996],
       [166.33931023],
       [ 96.61201385],
       [164.67655723],
       [183.00357809],
       [106.30745219],
       [131.23313567],
       [ 92.68518396],
       [126.09380255],
       [169.14425497],
       [ 43.07921018],
       [117.42105054],
       [134.47499936],
       [ 80.19479471],
       [ 97.19233509],
       [127.29499869],
       [146.81007071],
       [135.01720083],
       [140.56067961],
       [ 72.63342794],
       [174.29861108],
       [164.95604539],
       [161.06573252],
       [109.13252766],
       [ 90.82963836],
       [ 86.79187621],
       [100.83794263],
       [166.04898091],
       [ 90.56413683],
       [163.02804309],
       [ 71.4917088 ],
       [104.10232598],
       [131.13999519],
       [121.22052308],
       [159

In [37]:
# 거리와 등급의 행렬곱 연산이 왜 예측?
user_pred = traing_distance.dot(train_rating) / np.array([np.abs(traing_distance).sum(axis=1)]).T
user_pred, user_pred.shape


(array([[2.10259747e+00, 5.86975978e-01, 3.40264192e-01, ...,
         0.00000000e+00, 7.33611460e-03, 6.04379414e-03],
        [1.40999723e+00, 2.91863934e-01, 2.68085289e-01, ...,
         0.00000000e+00, 3.50378592e-03, 2.32963985e-03],
        [1.69014833e+00, 3.13648440e-01, 3.26127887e-01, ...,
         0.00000000e+00, 3.25391767e-03, 1.77210119e-03],
        ...,
        [1.73393747e+00, 4.06719333e-01, 3.21166908e-01, ...,
         0.00000000e+00, 2.71269625e-03, 9.00511411e-03],
        [2.34361031e+00, 8.10544770e-01, 4.73941025e-01, ...,
         0.00000000e+00, 1.01130066e-02, 9.66427605e-03],
        [2.36796969e+00, 5.98146138e-01, 3.85569804e-01, ...,
         0.00000000e+00, 6.39996638e-03, 5.37442746e-03]]),
 (660, 1682))

In [39]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, y):
    pred = pred[y.nonzero()].flatten()  # 0이 아닌 평점 예측값만 뽑음
    y = y[y.nonzero()].flatten()
    mse = mean_squared_error(pred,y)
    return mse

In [41]:
# train set mse
get_mse(user_pred,train_rating)

7.878218313143215

In [42]:
# test set mse
get_mse(user_pred,test_rating)

8.745164067978985

In [43]:
# 사용자가 많아지면 행렬 사이즈가 너무 커지는 문제점 발생
# KNN 방식 사용

# 비지도 방식 KNN , 기준 코사인 유사도
from sklearn.neighbors import NearestNeighbors

# 코사인 유사도 기준 가장 가까운 이웃 5개 추출
k = 5
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(train_rating)


NearestNeighbors(metric='cosine')

In [44]:
top_k_distance, top_k_users = knn.kneighbors(train_rating, return_distance=True)
top_k_distance, top_k_users

(array([[0.        , 0.49780748, 0.55784838, 0.56965585, 0.57163621],
        [0.        , 0.49511668, 0.53710425, 0.54977497, 0.55903851],
        [0.        , 0.66362987, 0.6641784 , 0.66766366, 0.6701703 ],
        ...,
        [0.        , 0.5764934 , 0.59340849, 0.59449253, 0.64699606],
        [0.        , 0.60496802, 0.6115226 , 0.62054374, 0.6229481 ],
        [0.        , 0.56320216, 0.60221688, 0.60314589, 0.6400121 ]]),
 array([[  0, 211,  16, 185, 101],
        [  1, 123, 484, 518, 262],
        [  2, 428,  39, 615, 211],
        ...,
        [657, 287, 271,  22, 258],
        [658, 407, 184, 618, 620],
        [659, 524, 230, 446, 632]]))

In [45]:
# top_k_users를 보면 0번째 유저와 가장 가까운 유저 5명은
# 0 , 211, 16, 185, 101을 알 수 있다.

# top_k_distance의 0행 1열 값은 0번째 유저와 211번째 유저의 거리


In [55]:
print(train_rating.shape)
print(top_k_distance.shape)
pred = np.zeros(train_rating.shape) # 660, 1682

for i in range(train_rating.shape[0]):
    # 거리와 정답간의 곱으로 예측값을 구한다.
    pred[i,:] = top_k_distance[i].dot(train_rating[top_k_users[i]]) \
        / np.arr/at
pred, pred.shape

(660, 1682)
(660, 5)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [9.19900165, 6.11607109, 4.35342362, ..., 0.        , 0.        ,
        0.        ],
       [4.70533591, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [50]:
top_k_distance.shape, train_rating.shape

((660, 5), (660, 1682))

In [48]:
train_rating[top_k_users[0]]

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [49]:
train_rating[top_k_users][0]

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])